# 프로젝트 : HuggingFace 커스텀 프로젝트 만들기

## 1. Processor 만들기

## 2. Tokenizer와 Model

## 3. 모델 학습하기

## 4. HuggingFace 활용하기

In [1]:
!pip install tensorflow-datasets -U

In [2]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from dataclasses import asdict
from transformers.data.processors.utils import DataProcessor, InputExample, InputFeatures

import warnings
warnings.filterwarnings('ignore')

import logging
logging.disable(logging.WARNING)

import transformers
transformers.logging.set_verbosity_error()

- - -
### Processor 만들기

In [3]:
data, info = tfds.load('glue/mnli', with_info=True)
info.splits['train'].num_examples

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/5 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/392702 [00:00<?, ? examples/s]

...

Shuffling /aiffel/tensorflow_datasets/glue/mnli/2.0.0.incompletePWDVPL/glue-test_matched.tfrecord*...:   0%|  …

Generating test_mismatched examples...:   0%|          | 0/9847 [00:00<?, ? examples/s]

Shuffling /aiffel/tensorflow_datasets/glue/mnli/2.0.0.incompletePWDVPL/glue-test_mismatched.tfrecord*...:   0%…

Dataset glue downloaded and prepared to /aiffel/tensorflow_datasets/glue/mnli/2.0.0. Subsequent calls will reuse this data.


392702

In [4]:
data

{Split('train'): <PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>,
 'validation_matched': <PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>,
 'validation_mismatched': <PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>,
 'test_matched': <PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>,
 'test_mismatched': <PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>}

`train`, `validation_matched`, `validation_mismatched`, `test_matched`, `test_mismatched`가 준비되어 있습니다.

In [5]:
info

tfds.core.DatasetInfo(
    name='glue',
    full_name='glue/mnli/2.0.0',
    description="""
    GLUE, the General Language Understanding Evaluation benchmark
    (https://gluebenchmark.com/) is a collection of resources for training,
    evaluating, and analyzing natural language understanding systems.
    """,
    config_description="""
    The Multi-Genre Natural Language Inference Corpus is a crowdsourced
    collection of sentence pairs with textual entailment annotations. Given a premise sentence
    and a hypothesis sentence, the task is to predict whether the premise entails the hypothesis
    (entailment), contradicts the hypothesis (contradiction), or neither (neutral). The premise sentences are
    gathered from ten different sources, including transcribed speech, fiction, and government reports.
    We use the standard test set, for which we obtained private labels from the authors, and evaluate
    on both the matched (in-domain) and mismatched (cross-domain) section. We a

대충 전제와 가설이 준비되어 있고 서로 수반하는지를 예측하는 데이터셋이라고 합니다

이렇게 데이터셋을 확인하고 있으니 Exploration 프로젝트 01을 했을 때가 떠오르네요.

In [6]:
data['train'].take(1)
# [hypothesis, idx, label, premise]

<TakeDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>

데이터셋의 shapes입니다. `premise`는 전제, `hypothesis`는 가설을 뜻합니다.

In [7]:
data['test_mismatched'].take(1)

<TakeDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>

저희가 예측할 테스트 데이터셋도 동일합니다.

In [8]:
examples = data['train'].take(5)

for example in examples:
    hypothesis = example['hypothesis']
    premise = example['premise']
    label = example['label']
    print("premise :", premise)
    print("hypothesis :", hypothesis)
    print("label :", label)
    print()

premise : tf.Tensor(b'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.', shape=(), dtype=string)
hypothesis : tf.Tensor(b'Meaningful partnerships with stakeholders is crucial.', shape=(), dtype=string)
label : tf.Tensor(1, shape=(), dtype=int64)

premise : tf.Tensor(b'The Clinton surrogates also held the high ground in the context war.', shape=(), dtype=string)
hypothesis : tf.Tensor(b'The Clinton followers kept to the higher ground in the discussion. ', shape=(), dtype=string)
label : tf.Tensor(0, shape=(), dtype=int64)

premise : tf.Tensor(b"um-hum because women are in every field now i mean i can't think of a field that they're not involved in", shape=(), dtype=string)
hypothesis : tf.Tensor(b'Women have jobs in all areas of the workforce, 

In [9]:
examples = data['validation_matched'].take(5)

for example in examples:
    hypothesis = example['hypothesis']
    premise = example['premise']
    label = example['label']
    print("premise :", premise)
    print("hypothesis :", hypothesis)
    print("label :", label)
    print()

premise : tf.Tensor(b'uh-huh oh yeah all the people for right uh life or something', shape=(), dtype=string)
hypothesis : tf.Tensor(b'yeah lots of people for the right life ', shape=(), dtype=string)
label : tf.Tensor(0, shape=(), dtype=int64)

premise : tf.Tensor(b'Also, I will be assuming that the 6.0a cost of the Postal Service to take the mail from basic to workshared condition is constant as limited quantities of mail move back and forth between basic and workshared.', shape=(), dtype=string)
hypothesis : tf.Tensor(b' I will be assuming that the 6.0a cost of the Postal Service to take the mail from basic to workshared condition is constant.', shape=(), dtype=string)
label : tf.Tensor(0, shape=(), dtype=int64)

premise : tf.Tensor(b"taken up by the oh okay oh so you know well that's i had wondered sometimes i knew that there was a lot of a lot of effort and a lot of work went into a lot of that and i just wondered if if it lasted and if it took you know like yeah", shape=(), dtype=

In [10]:
examples = data['validation_mismatched'].take(5)

for example in examples:
    hypothesis = example['hypothesis']
    premise = example['premise']
    label = example['label']
    print("premise :", premise)
    print("hypothesis :", hypothesis)
    print("label :", label)
    print()

premise : tf.Tensor(b'Projects which enliven and enrich the student experience and draw some of our finest scholars and teachers to our campus--and to our city.', shape=(), dtype=string)
hypothesis : tf.Tensor(b"These projects are largely ignored and don't impact anyone. ", shape=(), dtype=string)
label : tf.Tensor(2, shape=(), dtype=int64)

premise : tf.Tensor(b'The center also contacted the West Virginia State Police and asked whether any reports of a downed aircraft had been received.', shape=(), dtype=string)
hypothesis : tf.Tensor(b'The State Police in West Virginia were contacted regarding reports of fallen aircrafts.', shape=(), dtype=string)
label : tf.Tensor(0, shape=(), dtype=int64)

premise : tf.Tensor(b"Don Quixote is listed in the Appendix, but I could not find him in the dictionary, nor could I find quixotic, Queen of Sheba, Jeeves, Venus, Einstein (for `genius'), Hitler (for `demagogue, tryant'), Attila the Hun (for `barbarian'), Robinson Crusoe (for `castaway; lonely pe

In [11]:
examples = data['test_matched'].take(5)

for example in examples:
    hypothesis = example['hypothesis']
    premise = example['premise']
    label = example['label']
    print("premise :", premise)
    print("hypothesis :", hypothesis)
    print("label :", label)
    print()

premise : tf.Tensor(b'well is there a little electric pump you put in there', shape=(), dtype=string)
hypothesis : tf.Tensor(b'Is there a pump that you put in there?', shape=(), dtype=string)
label : tf.Tensor(-1, shape=(), dtype=int64)

premise : tf.Tensor(b"The Marquis de Sade's head is known to be above ground, as they say in the business.", shape=(), dtype=string)
hypothesis : tf.Tensor(b"The Marquis de Sade's head is safely kept underground.", shape=(), dtype=string)
label : tf.Tensor(-1, shape=(), dtype=int64)

premise : tf.Tensor(b'The Galilee Experience at the marina provides a 35-minute audio-visual introduction to the region; pleasure cruisers run to Capernaeum and other places; and you can also hire small craft.', shape=(), dtype=string)
hypothesis : tf.Tensor(b'Most visitors watch the video before they set out to explore the area.', shape=(), dtype=string)
label : tf.Tensor(-1, shape=(), dtype=int64)

premise : tf.Tensor(b"He'd better ask Dorcas, or one of the maids, if he 

In [12]:
examples = data['test_mismatched'].take(5)

for example in examples:
    hypothesis = example['hypothesis']
    premise = example['premise']
    label = example['label']
    print("premise :", premise)
    print("hypothesis :", hypothesis)
    print("label :", label)
    print()

premise : tf.Tensor(b'Anyway, I treasure that table.', shape=(), dtype=string)
hypothesis : tf.Tensor(b'That table has been in my family for generations.', shape=(), dtype=string)
label : tf.Tensor(-1, shape=(), dtype=int64)

premise : tf.Tensor(b'The Pentagon had been struck by American 77 at 9:37:46.', shape=(), dtype=string)
hypothesis : tf.Tensor(b"The Pentagon building was hit by an airplane after nine o'clock.", shape=(), dtype=string)
label : tf.Tensor(-1, shape=(), dtype=int64)

premise : tf.Tensor(b'After evaluating the law school building the I.U.', shape=(), dtype=string)
hypothesis : tf.Tensor(b'After evaluating the Harvard building.', shape=(), dtype=string)
label : tf.Tensor(-1, shape=(), dtype=int64)

premise : tf.Tensor(b'Our class has been challenged to support a program which will directly benefit the students at the IU School of Dentistry.', shape=(), dtype=string)
hypothesis : tf.Tensor(b'Our class is happy to support the program that benefits IU students.', shape=(

밸리데이션과 테스트 데이터셋은 `matched`와 `mismatched`가 있는데 정확히 무얼 뜻하는지는 모르겠습니다.

그리고 테스트 데이터셋은 레이블이 `-1`로만 이루어져 있습니다.

아무래도 아직 저희가 진짜 레이블을 확인할 수는 없고 따로 제공을 하는 것 같습니다.

In [13]:
class DataProcessor:
    """Base class for data converters for sequence classification data sets."""

    def get_example_from_tensor_dict(self, tensor_dict):
        """
        Gets an example from a dict with tensorflow tensors.

        Args:
            tensor_dict: Keys and values should match the corresponding Glue
                tensorflow_dataset examples.
        """
        raise NotImplementedError()

    def get_train_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the dev set."""
        raise NotImplementedError()

    def get_test_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the test set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    def tfds_map(self, example): # 만약 레이블의 종류가 한 개가 아니면 example.label을 get_labels에서 레이블 지정
        """
        Some tensorflow_datasets datasets are not formatted the same way the GLUE datasets are. This method converts
        examples to the correct format.
        """
        if len(self.get_labels()) > 1:
            example.label = self.get_labels()[int(example.label)]
        return example

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            return list(csv.reader(f, delimiter="\t", quotechar=quotechar))

# DataProcessor
#└ get_example_from_tensor_dict
#└ get_train_examples
#└ get_dev_examples
#└ get_test_examples
#└ get_labels 
#└ tfds_map : Args (example), return (example) 레이블 가공
# self._read_tsv : Args (input_file, quotechar), return (csv.reader)

`DataProcessor` 클래스입니다.

제가 이해한 바로는 여러 데이터셋을 상대할 수 있는 최고층 프로세서로 표현할 수 있을 것 같습니다.

때문에 딱히 수정할 부분도, 중요한 부분도 없는 것 같습니다.

In [14]:
class MnliProcessor(DataProcessor):
    """Processor for the Mnli data set (GLUE version)."""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def get_example_from_tensor_dict(self, tensor_dict): # [idx, premise, hypothesis, label]
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["premise"].numpy().decode("utf-8"),
            tensor_dict["hypothesis"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()), # str(label)
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        print("LOOKING AT {}".format(os.path.join(data_dir, "train.tsv")))
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")

    def get_labels(self):
        """See base class."""
        return ["0", "1", "2"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, i)
            text_a = line[3]
            text_b = line[4]
            label = None if set_type == "test" else line[0] # set_type이 "test"일 경우 label 제거
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples

# MnliProcessor(DataProcessor)
#└ __init__ : 부모 클래스를 읽어온다.
#└ get_example_from_tensor_dict : Args (tensor_dict), return (InputExample(idx, hypothesis, premise, label))
#└ self._create_examples : Args (lines, set_type), return (examples) 첫 번째 인덱스는 건너뛰고 [InputExample("set_type-i", hypothesis, premise, label)]
#└ get_train_examples : self._create_examples(train.tsv)
#└ get_dev_examples : self._create_examples(dev.tsv)
#└ get_test_examples : self._create_examples(test.tsv)
#└ get_labels : return (["0", "1", "2"])

`DataProcessor`의 하위 클래스입니다.

저희가 `MNLI` 데이터셋에 다룰 수 있게 수정해야 할 부분들이 보입니다.

`MNLI` 데이터셋을 다루는 게 목적이기 때문에 이름도 `MnliProcessor`로 변경했습니다.

예제에선 `get_example_from_tensor_dict`와 `get_labels`만 활용되기 때문에 나머지 메서드들은 어떤 상황에 필요한지 잘 모르겠습니다.

    가설이 먼저인가? 전제가 먼저인가? 고민해봤는데 전제가 먼저인 것 같습니다.

In [15]:
processor = MnliProcessor()
examples = data['train'].take(1)

for example in examples:
    print('------원본데이터------')
    print(example)  
    example = processor.get_example_from_tensor_dict(example)
    print('------processor 가공데이터------')
    print(example)

# processor.get_example_from_tensor_dict(dict) : InputExample 모듈로 가공(guid(idx), text_a(premise), text_b(hypothesis), label))

------원본데이터------
{'hypothesis': <tf.Tensor: shape=(), dtype=string, numpy=b'Meaningful partnerships with stakeholders is crucial.'>, 'idx': <tf.Tensor: shape=(), dtype=int32, numpy=16399>, 'label': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'premise': <tf.Tensor: shape=(), dtype=string, numpy=b'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.'>}
------processor 가공데이터------
InputExample(guid=16399, text_a='In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.', text_b='Meaningful partnerships with stakeh

`glue`에서 가져온 데이터셋을 제가 만든 프로세서에 넣으면 `InputExample(guid, text_a, text_b, label)`로 가공됩니다.

- - -
### Tokenizer와 Model

In [16]:
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification

tokenizer = DistilBertTokenizer.from_pretrained("typeform/distilbert-base-uncased-mnli")
model = TFDistilBertForSequenceClassification.from_pretrained("typeform/distilbert-base-uncased-mnli")

# tokenizer : typeform/distilbert-base-uncased-mnli
# model : typeform/distilbert-base-uncased-mnli

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/258 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/776 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

저는 토크나이저와 모델로 `typeform/distilbert-base-uncased-mnli`를 가져와줬습니다.

[Hugging Face Models](https://huggingface.co/models)에 있는 여러 모델들을 가져와봤지만 정상적으로 작동되는 모델은 이것 뿐이였습니다.

    제가 쓸 줄 몰라서 그런걸겁니다.

In [17]:
def _glue_convert_examples_to_features(examples, tokenizer, processor, max_length=None, label_list=None, output_mode="claasification") :
    if max_length is None : # max_length가 None이면 max_length를 tokenizer에서 찾음
        max_length = tokenizer.max_len
    if label_list is None:  # label_list가 None이면 label_list를 processor에서 에서 찾음
        label_list = processor.get_labels()
        print("Using label list %s" % (label_list))

    label_map = {label: i for i, label in enumerate(label_list)} # {'0': 0, '1': 1, '2': 2}
    labels = [label_map[example.label] for example in examples]  # examples의 labels 인코딩(원래 str 상태)

    batch_encoding = tokenizer( # {"input_ids" : Token Emb 배열, "attention_mask" : pad masking 배열, "token_type_ids" : Segment Emb 배열}
        [(example.text_a, example.text_b) for example in examples], # examples의 [(premise, hypothesis), ...] 배열
        max_length=max_length,
        padding="max_length",
        truncation=True,
    )
    
    features = []
    for i in range(len(examples)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding} # 각 examples 인덱스의 {input_ids, attention_mask, token_type_ids}

        feature = InputFeatures(**inputs, label=labels[i]) # InputFeatures(input_ids, attention_mask, token_type_ids, label)
        features.append(feature)

    for i, example in enumerate(examples[:5]):
        print("*** Example ***")
        print("guid: %s" % (example.guid)) # id
        print("features: %s" % features[i])

    return features

In [18]:
def tf_glue_convert_examples_to_features(examples, tokenizer, processor, max_length=None, label_list=None, output_mode="classification") :
    """
    :param examples: tf.data.Dataset
    :param tokenizer: pretrained tokenizer
    :param max_length: example의 최대 길이(기본값 : tokenizer의 max_len)
    :param task: GLUE task 이름
    :param label_list: 라벨 리스트
    :param output_mode: "regression" or "classification"

    :return: task에 맞도록 feature가 구성된 tf.data.Dataset
    """
    examples = [processor.tfds_map(processor.get_example_from_tensor_dict(example)) for example in examples]
            # [InputExample(idx, premise, hypothesis, label) for examples]
    features = _glue_convert_examples_to_features(examples, tokenizer, processor, max_length)
            # [InputFeatures(input_ids, attention_mask, token_type_ids, label) for examples]
    label_type = tf.int64

    def gen():
        for ex in features: # features 저장
            d = {k: v for k, v in asdict(ex).items() if v is not None} # 데이터 클래스로 변경
            label = d.pop("label") # label 분리
            yield (d, label) # d와 label 전달

    input_names = ["input_ids"] + tokenizer.model_input_names # ['input_ids', 'input_ids', 'attention_mask', 'token_type_ids'} ?

    return tf.data.Dataset.from_generator( # 데이터셋 생성
        gen,
        ({k: tf.int32 for k in input_names}, label_type),
        ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
    )

이번에도 features 가공 함수를 `mnli` 태스크에 맞게 바꿔줍니다.

In [19]:
train_dataset = tf_glue_convert_examples_to_features(data['train'], tokenizer, processor=processor, max_length=64)

Using label list ['0', '1', '2']
*** Example ***
guid: 16399
features: InputFeatures(input_ids=[101, 1999, 5038, 1997, 2122, 13136, 1010, 1048, 11020, 2038, 2499, 29454, 29206, 14626, 2144, 2786, 2000, 16636, 1996, 10908, 1997, 1996, 2110, 4041, 6349, 1998, 2000, 5323, 15902, 13797, 2007, 22859, 6461, 2012, 6469, 2075, 1037, 2047, 25353, 14905, 10735, 2483, 2090, 1996, 2976, 10802, 1998, 15991, 1997, 3423, 2578, 4804, 1012, 102, 15902, 13797, 2007, 22859, 2003, 10232, 1012, 102, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], token_type_ids=None, label=1)
*** Example ***
guid: 206287
features: InputFeatures(input_ids=[101, 1996, 7207, 7505, 21799, 2015, 2036, 2218, 1996, 2152, 2598, 1999, 1996, 6123, 2162, 1012, 102, 1996, 7207, 8771, 2921, 2000, 1996, 3020, 2598, 1999, 1996, 6594, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

함수에 넣은 데이터는 불러온 토크나이저로 토큰화된 후 `guid`와 `features`로 추출됩니다.

`features`는 `input_ids`, `attention_mask`, `token_type_ids`, `label`로 구성되어 있습니다.

`input_ids`는 토큰 임베딩, `attention_mask`는 0으로 패딩된 부분을 무시하기 위한 임베딩이며, `label`은 레이블,  
`token_type_ids`는 하나로 합쳐진 두 문장을 구별하기 위한 임베딩으로 지금의 태스크에서 필요한 임베딩이지 않나 싶은데 

    Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.

오류가 나며 무시됩니다.

In [20]:
examples = train_dataset.take(1)

for example in examples:
    print(example)

({'input_ids': <tf.Tensor: shape=(64,), dtype=int32, numpy=
array([  101,  1999,  5038,  1997,  2122, 13136,  1010,  1048, 11020,
        2038,  2499, 29454, 29206, 14626,  2144,  2786,  2000, 16636,
        1996, 10908,  1997,  1996,  2110,  4041,  6349,  1998,  2000,
        5323, 15902, 13797,  2007, 22859,  6461,  2012,  6469,  2075,
        1037,  2047, 25353, 14905, 10735,  2483,  2090,  1996,  2976,
       10802,  1998, 15991,  1997,  3423,  2578,  4804,  1012,   102,
       15902, 13797,  2007, 22859,  2003, 10232,  1012,   102,     0,
           0], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(64,), dtype=int32, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
      dtype=int32)>}, <tf.Tensor: shape=(), dtype=int64, numpy=1>)


함수를 거쳐 나온 데이터셋도 기존 데이터셋의 shapes를 가지고 있습니다.

함수에 있던 dataclasses.asdict의 영향인 것 같습니다.

In [21]:
train_dataset_batch = train_dataset.shuffle(100).batch(16).repeat(2)

In [22]:
# validation 데이터셋
validation_dataset = tf_glue_convert_examples_to_features(data['validation_mismatched'], tokenizer, max_length=64, processor=processor)
validation_dataset_batch = validation_dataset.shuffle(100).batch(16)

Using label list ['0', '1', '2']
*** Example ***
guid: 9410
features: InputFeatures(input_ids=[101, 3934, 2029, 4372, 3669, 8159, 1998, 4372, 13149, 1996, 3076, 3325, 1998, 4009, 2070, 1997, 2256, 10418, 5784, 1998, 5089, 2000, 2256, 3721, 1011, 1011, 1998, 2000, 2256, 2103, 1012, 102, 2122, 3934, 2024, 4321, 6439, 1998, 2123, 1005, 1056, 4254, 3087, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=None, label=2)
*** Example ***
guid: 4506
features: InputFeatures(input_ids=[101, 1996, 2415, 2036, 11925, 1996, 2225, 3448, 2110, 2610, 1998, 2356, 3251, 2151, 4311, 1997, 1037, 20164, 2948, 2018, 2042, 2363, 1012, 102, 1996, 2110, 2610, 1999, 2225, 3448, 2020, 11925, 4953, 4311, 1997, 5357, 2948, 2015, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [23]:
# test 데이터셋
test_dataset = tf_glue_convert_examples_to_features(data['test_mismatched'], tokenizer, max_length=64, processor=processor)
test_dataset_batch = test_dataset.shuffle(100).batch(16)

Using label list ['0', '1', '2']
*** Example ***
guid: 3498
features: InputFeatures(input_ids=[101, 4312, 1010, 1045, 8813, 2008, 2795, 1012, 102, 2008, 2795, 2038, 2042, 1999, 2026, 2155, 2005, 8213, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=None, label=2)
*** Example ***
guid: 5295
features: InputFeatures(input_ids=[101, 1996, 20864, 2018, 2042, 4930, 2011, 2137, 6255, 2012, 1023, 1024, 4261, 1024, 4805, 1012, 102, 1996, 20864, 2311, 2001, 2718, 2011, 2019, 13297, 2044, 3157, 1051, 1005, 5119, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

`train`과 `validation`, `test` 데이터셋을 모두 준비해줍니다.

밸리데이션과 테스트 데이터셋은 `mismatched`와 `matched` 중 `mismatched`를 사용했습니다.

둘 다 사용해본 결과 `matched`는 학습이 진행되지 않아서 loss가 고정되는 현상이 있습니다.

    이유는 아직 모르겠습니다.

- - -
### 모델 학습하기

In [24]:
model

제가 가져온 모델을 확인해봅니다.

In [25]:
model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["acc"],
)

model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  66362880  
_________________________________________________________________
pre_classifier (Dense)       multiple                  590592    
_________________________________________________________________
classifier (Dense)           multiple                  2307      
_________________________________________________________________
dropout_19 (Dropout)         multiple                  0         
Total params: 66,955,779
Trainable params: 66,955,779
Non-trainable params: 0
_________________________________________________________________


여기서도 [Huigging Face](https://huggingface.co/models)의 일부 모델은 `model.compile`을 할 수 없는 모델이였습니다.  
이것도 아마 제가 쓸 줄 몰라서 그런걸겁니다.

그리고 레이어의 최종 출력도 `Dropout`인 것도 의아스럽습니다.

model의 output을 따로 지정해줘야 하는 게 아닌가 싶기도 한데 단순한 방법으론 적용이 안돼서 포기했습니다.

In [26]:
# 이전 스텝에서 배치처리를 진행한 데이터셋(xxxx_dataset_batch)을 활용
model.fit(train_dataset_batch, epochs=5, steps_per_epoch=115,
                validation_data=validation_dataset_batch)

Epoch 1/5
115/115 [==============================] - 53s 373ms/step - loss: 0.2147 - acc: 0.9364 - val_loss: 0.6604 - val_acc: 0.8054
Epoch 2/5
115/115 [==============================] - 42s 367ms/step - loss: 0.1887 - acc: 0.9413 - val_loss: 0.6481 - val_acc: 0.7992
Epoch 3/5
115/115 [==============================] - 41s 362ms/step - loss: 0.2011 - acc: 0.9293 - val_loss: 0.6681 - val_acc: 0.8067
Epoch 4/5
115/115 [==============================] - 42s 365ms/step - loss: 0.2273 - acc: 0.9201 - val_loss: 0.6218 - val_acc: 0.8037
Epoch 5/5
115/115 [==============================] - 42s 366ms/step - loss: 0.2206 - acc: 0.9212 - val_loss: 0.6342 - val_acc: 0.8042


배치 사이즈는 16인데 `steps_per_epoch`는 115, 총 5번의 학습.

39만개의 데이터인 것에 비하면 굉장히 짧은 학습이였습니다.

그래도 loss는 이미 아니, 진작에 0.2에 수렴되어 있었습니다.

pre-train 되어있다는 게 이런 의미겠죠?

In [27]:
result = model.evaluate(test_dataset_batch)
print(result)

616/616 [==============================] - 28s 45ms/step - loss: 3.6705 - acc: 0.3058
[3.670511484146118, 0.3057784140110016]


하지만 test 데이터셋에 대한 정확도는 1/3입니다. 이는 정답을 모두 찍은 수준입니다.

제가 여기서 의문이 드는 것은 테스트 데이터셋의 레이블은 모두 `-1`이였습니다. 그래서 정확도가 0이 나올거라 예상했는데 말이죠.

예상가는 부분이 있습니다.

In [28]:
examples = test_dataset.take(10)

for example in examples:
    print(example)

({'input_ids': <tf.Tensor: shape=(64,), dtype=int32, numpy=
array([ 101, 4312, 1010, 1045, 8813, 2008, 2795, 1012,  102, 2008, 2795,
       2038, 2042, 1999, 2026, 2155, 2005, 8213, 1012,  102,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(64,), dtype=int32, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)>}, <tf.Tensor: shape=(), dtype=int64, numpy=2>)
({'input_ids': <tf.Tensor: shape=(64,), dtype=int32, numpy=
array([  101,  1996, 20864,  2018,  2042,  4930,  2011,  2137,  6255,
        2012,  1023,  10

그렇습니다.

레이블 가공을 거쳤기 때문에 존재하지 않는 `-1`이란 레이블은 `['0', '1', '2']`의 -1 인덱스에 해당하는 `2`로 변경됐던 것입니다.

그리고 두 문장이 수반하는지에 상관없이 무조건 2라는 레이블을 가지니 정확도가 1/3이였던 거구요.

In [29]:
output_dir = os.getenv('HOME') + '/aiffel/transformers'
output_eval_file = os.path.join(output_dir, "eval_results.txt")

with open(output_eval_file, "w") as writer:
    writer.write("Loss = %f\t" %(result[0]))
    writer.write("Accuracy = %f\n" %(result[1]))

#파일에 쓴 테스트 결과 확인
!cat ~/aiffel/transformers/eval_results.txt

Loss = 3.670511	Accuracy = 0.305778


어찌됐든 저장까지 하는 것으로 끝입니다.

- - -
### HuggingFace 활용하기

In [30]:
import datasets
from datasets import load_dataset

huggingface_mnli_dataset = load_dataset('glue', 'mnli')
print(huggingface_mnli_dataset)

# DatasetDict
#└ train : {features, num_rows}
#└ validation_matched : {features, num_rows}
#└ validation_mismatched : {features, num_rows}
#└ test_matched : {features, num_rows}
#└ test_mismatched : {features, num_rows}

Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/313M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset glue downloaded and prepared to /aiffel/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/5 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})


`datasets`에서 데이터셋을 불러옵니다.

In [31]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

huggingface_tokenizer = AutoTokenizer.from_pretrained("typeform/distilbert-base-uncased-mnli")
huggingface_model = AutoModelForSequenceClassification.from_pretrained("typeform/distilbert-base-uncased-mnli")

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

`AutoTokenizer`와 `AutoModelForSequenceClassification`을 이용해 토크나이저와 모델을 불러옵니다.

In [32]:
def transform(data):
    return tokenizer( # return {input_ids, attention_mask, # Segment Emb}
        data['premise'],
        data['hypothesis'],
        truncation = True, # 특정 문장이 길 경우 짧게 짜를 것인지
        padding = 'max_length',
        return_token_type_ids = True, # Segment Emb
        )
  
examples = huggingface_mnli_dataset['train'][:5]
examples_transformed = transform(examples)

print(examples)
print(examples_transformed)

{'premise': ['Conceptually cream skimming has two basic dimensions - product and geography.', 'you know during the season and i guess at at your level uh you lose them to the next level if if they decide to recall the the parent team the Braves decide to call to recall a guy from triple A then a double A guy goes up to replace him and a single A guy goes up to replace him', 'One of our number will carry out your instructions minutely.', 'How do you know? All this is their information again.', "yeah i tell you what though if you go price some of those tennis shoes i can see why now you know they're getting up in the hundred dollar range"], 'hypothesis': ['Product and geography are what make cream skimming work. ', 'You lose the things to the following level if the people recall.', 'A member of my team will execute your orders with immense precision.', 'This information belongs to them.', 'The tennis shoes have a range of prices.'], 'label': [1, 0, 0, 0, 1], 'idx': [0, 1, 2, 3, 4]}
{'inp

여기까지만 해도 `tf_glue_convert_examples_to_features` 함수를 만들던 과정까지 완료된겁니다.

In [33]:
encoded_dataset = huggingface_mnli_dataset.map(transform, batched=True)

  0%|          | 0/393 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [34]:
#메모리를 비워줍니다.
del model
del tokenizer
del train_dataset_batch
del validation_dataset_batch
del test_dataset_batch

In [35]:
# Trainer을 활용하는 형태로 모델 재생성
from transformers import Trainer, TrainingArguments
output_dir = os.getenv('HOME') + '/aiffel/transformers'
metric_name = 'accuracy'

training_arguments = TrainingArguments(
    output_dir, # output이 저장될 경로
    evaluation_strategy="epoch", #evaluation하는 빈도
    learning_rate = 2e-5, #learning_rate
    per_device_train_batch_size = 16, # 각 device 당 batch size
    per_device_eval_batch_size = 16, # evaluation 시에 batch size
    num_train_epochs = 1, # train 시킬 총 epochs
    weight_decay = 0.01, # weight decay
)

In [36]:
from datasets import load_metric
metric = load_metric('glue', 'mnli')

def compute_metrics(eval_pred):    # 내부 함수 지정
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels)

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

In [37]:
trainer = Trainer(
    model=huggingface_model,                    # 학습시킬 model
    args=training_arguments,                    # TrainingArguments을 통해 설정한 arguments
    train_dataset=encoded_dataset['train'],     # training dataset
    eval_dataset=encoded_dataset['validation_mismatched'], # evaluation dataset
    compute_metrics=compute_metrics,
)

trainer.train()

{'loss': 0.1935, 'learning_rate': 1.9592568448500654e-05, 'epoch': 0.02}
{'loss': 0.1773, 'learning_rate': 1.9185136897001307e-05, 'epoch': 0.04}
{'loss': 0.2183, 'learning_rate': 1.8777705345501956e-05, 'epoch': 0.06}
{'loss': 0.2137, 'learning_rate': 1.837027379400261e-05, 'epoch': 0.08}
{'loss': 0.2198, 'learning_rate': 1.796284224250326e-05, 'epoch': 0.1}
{'loss': 0.218, 'learning_rate': 1.7555410691003914e-05, 'epoch': 0.12}
{'loss': 0.2233, 'learning_rate': 1.7147979139504566e-05, 'epoch': 0.14}
{'loss': 0.2105, 'learning_rate': 1.6740547588005215e-05, 'epoch': 0.16}
{'loss': 0.2254, 'learning_rate': 1.6333116036505868e-05, 'epoch': 0.18}
{'loss': 0.215, 'learning_rate': 1.592568448500652e-05, 'epoch': 0.2}
{'loss': 0.2294, 'learning_rate': 1.5518252933507173e-05, 'epoch': 0.22}
{'loss': 0.2096, 'learning_rate': 1.5110821382007822e-05, 'epoch': 0.24}
{'loss': 0.2192, 'learning_rate': 1.4703389830508477e-05, 'epoch': 0.26}
{'loss': 0.2127, 'learning_rate': 1.4295958279009128e-05, 

TrainOutput(global_step=24544, training_loss=0.20887737825143757, metrics={'train_runtime': 20505.4782, 'train_samples_per_second': 19.151, 'train_steps_per_second': 1.197, 'train_loss': 0.20887737825143757, 'epoch': 1.0})

이번엔 39만개의 데이터를 모두 사용해서 학습하니 1번의 학습만으로도 상당히 많은 시간이 소요됩니다.

In [38]:
trainer.evaluate(encoded_dataset['test_mismatched'])

/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [0,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [1,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [2,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [3,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [4,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [5,0,0] Assertion `t >= 0 

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

테스트 데이터셋 평가는 오류가 나지만 어차피 테스트 데이터셋의 레이블은 `-1` 뿐이라 의미는 없으니 그냥 여기서 마무리하겠습니다.

- - -
## 마무리하며

https://huggingface.co/models 에 가면 모델들이 LMS의 예제랑은 다른 예시를 가지고 있고,

원리를 찾아서 하기엔 LMS는 설명보다는 사용법만을 알려주고 있어서 상당히 힘들었습니다.

지금도 제가 제대로 한 건지 모르겠네요.

실패했던 pre-train 모델들을 사용하는 방법을 내가 못찾았다거나 데이터셋의 레이블을 불러올 수 있는 방법이 따로 있다던가 하는 찝찝함이 남아있습니다.

Processor도 제가 이해한 만큼 정리하긴 했지만 InputExample이나 InputFeatures나 asdict같은 함수는 중요해보여도 결국 뭔지 모른 채 넘어갔습니다.

https://huggingface.co/docs/transformers/index 에서 볼 수 있는 글은 전반적인 내용도 파악하지 못했습니다.

프로젝트 지시 사항에선 BERT 쓰지 말라고 했었는데요. 모델들은 전부 BERT였습니다.

프로젝트에서 말하는 BERT가 LMS의 예제에서 사용한 BERT를 쓰지 말라는 것인지 BERT가 활용된 모델을 쓰지 말라는 것인지 헷갈리게 하는 프로젝트였네요.